In [1]:
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaR, deltaPhi

import CMS_lumi, tdrstyle
a = tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2


# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [18]:
fpath =OrderedDict()
tree = OrderedDict()


# start_t = time.time()



scale = ['xiO_2p5_xiL_1', 'xiO_2p5_xiL_2p5', 'xiO_1_xiL_1']
prod = ['ggH']
portal = {
'higgs': [4, 5, 10, 15, 20],
'gluon': [3, 5, 10, 15, 20],
'vector':[2, 5, 10, 15, 20],
'photon':[2, 5, 10, 15, 20],
'darkphoton':[2, 5, 10, 15, 20],
}
old_ctau = [500, 1000, 5000, 10000]

ntupler_version = 'V1p17/'
analyzer_version = 'v2/v183/'

CTAU = 5000
MASS = 15
# PORTAL = 'higgs'
mc_path = {}
mc_path= '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
for p in portal.keys():
    for x in scale:
        for m in portal[p]:
            for ct in old_ctau:
                if not ct == CTAU:continue
                if not m == MASS:continue
#                 if not p == PORTAL:continue
#                 if portal == 'vector' and not scale == 'xiO_1_xiL_1':continue
#                 key = 'HV_params_'+p+'_m_'+str(m) + '_ctau_'+str(ct)+'mm_'+x
                key = 'HV_params_'+p+'_m_'+str(m) + '_ctau_'+str(ct)+'mm_'+x
                fpath[key] = mc_path+key+'.root'
                
# fpath['DarkShowerHiggs_darkphoton_M15_pl1000_XIOMEGA1_XILAMBDA1_1pb_weighted'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p19/MC_Fall18/v1/v178/normalized/DarkShowerHiggs_darkphoton_M15_pl1000_XIOMEGA1_XILAMBDA1_1pb_weighted.root'
# fpath['DarkShowerHiggs_darkphoton_M15_pl100_XIOMEGA1_XILAMBDA1_1pb_weighted'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p19/MC_Fall18/v1/v178/normalized/DarkShowerHiggs_darkphoton_M15_pl100_XIOMEGA1_XILAMBDA1_1pb_weighted.root'
# fpath['DarkShowerHiggs_darkphoton_M2_pl100_XIOMEGA1_XILAMBDA1_1pb_weighted'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p19/MC_Fall18/v1/v178/normalized/DarkShowerHiggs_darkphoton_M2_pl100_XIOMEGA1_XILAMBDA1_1pb_weighted.root'
# fpath['DarkShowerHiggs_darkphoton_M2_pl1000_XIOMEGA1_XILAMBDA1_1pb_weighted'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p19/MC_Fall18/v1/v178/normalized/DarkShowerHiggs_darkphoton_M2_pl1000_XIOMEGA1_XILAMBDA1_1pb_weighted.root'




NEvents = {}
Total = {}
accep = {}
accep_met = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    try:
        tree[k] = root_dir['MuonSystem']
        NEvents[k] = root_dir['NEvents'][1]
        a = tree[k]["weight"].array()
    except:
        print('NOT FOUND', v)

NOT FOUND /storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//MC_all/v2/v183//normalized/HV_params_vector_m_15_ctau_5000mm_xiO_2p5_xiL_1.root
NOT FOUND /storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//MC_all/v2/v183//normalized/HV_params_vector_m_15_ctau_5000mm_xiO_2p5_xiL_2p5.root


In [19]:
for k in tree.keys():
    print(k)

HV_params_higgs_m_15_ctau_5000mm_xiO_2p5_xiL_1
HV_params_higgs_m_15_ctau_5000mm_xiO_2p5_xiL_2p5
HV_params_higgs_m_15_ctau_5000mm_xiO_1_xiL_1
HV_params_gluon_m_15_ctau_5000mm_xiO_2p5_xiL_1
HV_params_gluon_m_15_ctau_5000mm_xiO_2p5_xiL_2p5
HV_params_gluon_m_15_ctau_5000mm_xiO_1_xiL_1
HV_params_vector_m_15_ctau_5000mm_xiO_1_xiL_1
HV_params_photon_m_15_ctau_5000mm_xiO_2p5_xiL_1
HV_params_photon_m_15_ctau_5000mm_xiO_2p5_xiL_2p5
HV_params_photon_m_15_ctau_5000mm_xiO_1_xiL_1
HV_params_darkphoton_m_15_ctau_5000mm_xiO_2p5_xiL_1
HV_params_darkphoton_m_15_ctau_5000mm_xiO_2p5_xiL_2p5
HV_params_darkphoton_m_15_ctau_5000mm_xiO_1_xiL_1


# signal

In [20]:
# for p in portal.keys():
#     for m in portal[p]:
#         if not m == MASS:continue
#         for x in scale:
#             for ct in old_ctau:
#                 if not ct == CTAU:continue
#                 k = 'HV_params_'+p+'_m_'+str(m) + '_ctau_'+str(ct)+'mm_'+x
#                 if not k == 'HV_params_higgs_m_15_ctau_5000mm_xiO_2p5_xiL_2p5':continue

#                 if k not in tree.keys():continue
#                 print('selections_sig_'+k)
for k in tree.keys():
    T = tree[k]
    sel_ev = {}
    weight = {}
    dphiMet_cluster = {}
    bdt_sel = {}
    Nrechits = {}
    ncluster = 1
    cluster_index = ''
    table = OrderedDict()
    table['met'] = "Trigger and MET "
    table['met_filter'] = "MET filters "
    table['jet'] = r"$N_{jet} \geq 1$ "
    table['cosmic_veto'] = r"$N_{\text{CSC+DT rings}} \leq 10 $ "
    table['spike'] = "DT noise veto"
    table['2_cluster'] = "$N_{cluster} \geq 2$ "
    table['dt_cosmic_veto'] = "DT cosmic veto"
    table['muon_veto'] = "muon veto "
    table['jet_veto'] = "jet veto "
    table['me11_ratio'] = "ME11/MB1 ratio "
    table['station_ratio'] = r"$N_{\mathrm{min station hits}}$/$N_{\mathrm{max station hits}}$ "
    table['time'] = "time cut "
    table['time_spread'] = "time spread "
    table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
    table['deltaR'] = r"$\Delta R \mathrm{ (cluster1,cluster2)}$ "
    table['nrechits'] = r"$N_{rechits}$ cut "



    for category in [0, 1, 2]:
        if category == 0:
            NRECHIT1 = 100
            NRECHIT2 = 100
        elif category == 1:
            NRECHIT1 = 80
            NRECHIT2 = 80
        else:
            NRECHIT1 = 80
            NRECHIT2 = 100

        weight = (T.array('pileupWeight')*T.array('metSF')*T.array('higgsPtWeight'))*48.58/2000000

        weight[T.array('MC_condition')==2016] *= 35920 
        weight[T.array('MC_condition')==2017] *= 41530 
        weight[T.array('MC_condition')==2018] *= 59740 
        ########### SELECTION: JETS ############

        sel_jet = np.logical_and(T.array('jetPt') >= 30, np.abs(T.array('jetEta')) < 2.4 )
        sel_jet = np.logical_and(sel_jet, T.array('jetTightPassId'))

        ########### SELECTION: EVENTS ############
        total = 48.58*137000

        sel_ev[k] = np.logical_and(T.array('metEENoise')>=200,T.array('METNoMuTrigger'))

#         sel_ev[k] = T.array('metEENoise')>=100

        acc_met = np.sum(weight[sel_ev[k]])


#                     print("trigger + MET", accep_met[k]/acceptance, np.sum(weight[sel_ev[k]])/sel_temp, accep_met[k]/total,np.count_nonzero(sel_ev[k]))
        table["met"] += " & {0:6.3f} & {1:6.3f}".format(100*acc_met/total, 100*acc_met/total)
        sel_temp = np.sum(weight[sel_ev[k]])



        sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
    #     print("MET filters", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
        table["met_filter"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
        sel_temp = np.sum(weight[sel_ev[k]])


        sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    #     print("1 jet", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
        table["jet"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
        sel_temp = np.sum(weight[sel_ev[k]])



        sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtRings")+T.array("nCscRings") < 10)
    #     print("cosmic shower veto", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
        table["cosmic_veto"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

        sel_temp = np.sum(weight[sel_ev[k]])



        spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
        spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
        spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)
        sel_ev[k] = np.logical_and(sel_ev[k] , np.logical_not(spike))
    #     print("spike", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
        table["spike"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

        sel_temp = np.sum(weight[sel_ev[k]])


        ########### SELECTION: CLUSTERS ############

        if category == 0:
            sel_rechitcluster = T.array('cscRechitCluster' + cluster_index + 'Size')>=50
#             sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_gLLP'))
    #         print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["2_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


            sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 100, T.array('cscRechitClusterMuonVetoGlobal'))))
    #         print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp) 
            table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


            sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < 30)
    #         print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["jet_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

#                         b = create_TH1D(np.ones(len(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])), 'hm1', binning=[1,0,1000], weights = weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


            sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio') < 1)
    #         print("me11 ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["me11_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
#                         m = create_TH1D(np.ones(len(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])), 'hm1', binning=[1,0,1000], weights = weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
#                         eff = rt.TEfficiency(m,b)
#                         print(k, category, 'me11', np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]**2)**0.5/sel_temp, len(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))
#                         print(k, category, 'me11', eff.GetEfficiency(1), eff.GetEfficiencyErrorLow(1),  eff.GetEfficiencyErrorUp(1))

#             print(np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)))

            time = T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')[sel_rechitcluster][np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]
            sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5.0))
    #         print("time cut", np.sum([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
#             print(np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)))
            table["time"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


            sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll') < 20)
    #         print("time spread cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["time_spread"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])



            sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1.2)
    #         print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["dphi"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

            temp = np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)
            cond = deltaR(T.array('cscRechitClusterEta')[sel_rechitcluster][temp][:,0], T.array('cscRechitClusterPhi')[sel_rechitcluster][temp][:,0],
                                                         T.array('cscRechitClusterEta')[sel_rechitcluster][temp][:,1], T.array('cscRechitClusterPhi')[sel_rechitcluster][temp][:,1])<2
            sel_ev[k][temp] = cond
    #         print("deltaR cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

            table["deltaR"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

            sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'Size')) >= 100)
    #         print("nrechits cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

            table['station_ratio'] += "&  /  & /"
            table['dt_cosmic_veto'] += "&  /  & /"

        if category == 1:
            sel_rechitcluster = T.array('dtRechitCluster' + cluster_index + 'Size')>=0
#                         sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster' + cluster_index + '_match_gLLP_dt'))

    #         print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

            table["2_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


            sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                       T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
            nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
            +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

            sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))    
    #         print("dt cosmic veto cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["dt_cosmic_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])




            sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
    #         print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


            sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitCluster' + cluster_index + 'JetVetoPt') < 50)
    #         print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["jet_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
#                         b = create_TH1D(np.ones(len(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])), 'hm1', binning=[1,0,1000], weights = weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

            sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))
    #         print("MB1 ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["me11_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
#                         m = create_TH1D(np.ones(len(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])), 'hm1', binning=[1,0,1000], weights = weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
#                         eff = rt.TEfficiency(m,b)
#                         print(k, category, 'me11', np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]**2)**0.5/sel_temp, len(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))
#                         print(k, category, 'me11', eff.GetEfficiency(1), eff.GetEfficiencyErrorLow(1),  eff.GetEfficiencyErrorUp(1))

            cut = 5
            station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
        +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)
            max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
            min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
            sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
    #         print("min/max station ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["station_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


            sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1)
    #         print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["dphi"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

            sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'Size')) >= 80)
    #         print("nrechits cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
            table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                          100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()==2)])
    #         print(np.count_nonzero(sel_temp))

            table['time'] += " & / & / "
            table['time_spread'] += " & / & /"

            table['deltaR'] += " & / & /"


        if category == 2:
            sel_csc = T.array('cscRechitCluster' + cluster_index + 'Size')>=50
            sel_dt = T.array('dtRechitCluster' + cluster_index + 'Size')>=50
#                         sel_csc = np.logical_and(sel_csc, T.array('cscRechitCluster' + cluster_index + '_match_gLLP'))
#                         sel_dt = np.logical_and(sel_dt, T.array('dtRechitCluster' + cluster_index + '_match_gLLP'))


    #         print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["2_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])




            sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                       T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
            nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
            +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

            sel_dt = np.logical_and(sel_dt, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))    
    #         print("dt cosmic veto cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

            table["dt_cosmic_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])





            sel_dt = np.logical_and( sel_dt, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
            sel_csc = np.logical_and( sel_csc, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 100, T.array('cscRechitClusterMuonVetoGlobal'))))
    #         print("muon veto", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


            sel_dt = np.logical_and( sel_dt, T.array('dtRechitCluster' + cluster_index + 'JetVetoPt') < 50)
            sel_csc = np.logical_and(sel_csc, T.array('cscRechitClusterJetVetoPt') < 30)
    #         print("jet veto", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["jet_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

#                         b = create_TH1D(np.ones(len(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])), 'hm1', binning=[1,0,1000], \
#                                         weights = weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

            sel_csc = np.logical_and( sel_csc, T.array('cscRechitClusterMe11Ratio') < 1)
            sel_dt = np.logical_and(sel_dt, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))
    #         print("mb1 ratio", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["me11_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])
#                         m = create_TH1D(np.ones(len(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])), 'hm1', binning=[1,0,1000], \
#                                         weights = weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])
#                         eff = rt.TEfficiency(m,b)
#                         print(k, category, 'me11', \
#                               np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]**2)**0.5/sel_temp, \
#                               len(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))
#                         print(k, category, 'me11', eff.GetEfficiency(1), eff.GetEfficiencyErrorLow(1),  eff.GetEfficiencyErrorUp(1))


            cut = 5
            station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
        +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)
            max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
            min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
            sel_dt = np.logical_and(sel_dt, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
    #         print("station cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["station_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])



            sel_csc = np.logical_and(sel_csc, np.logical_and(T.array('cscRechitClusterTimeWeighted') < 12.5, T.array('cscRechitClusterTimeWeighted') > -5.0))
    #         print("time cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
            table["time"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


            sel_csc = np.logical_and(sel_csc, T.array('cscRechitClusterTimeSpreadWeightedAll')<20)
    #         print("time spread", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
            table["time_spread"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


            sel_dt = np.logical_and( sel_dt, np.abs(T.array('dtRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1)
            sel_csc = np.logical_and( sel_csc, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1.2)
    #         print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["dphi"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

            temp = np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))
            cond = deltaR(T.array('dtRechitClusterEta')[sel_dt][temp][:,0], T.array('dtRechitClusterPhi')[sel_dt][temp][:,0],
                                                         T.array('cscRechitClusterEta')[sel_csc][temp][:,0], T.array('cscRechitClusterPhi')[sel_csc][temp][:,0])<2.5

            sel_ev[k][temp] = cond

    #         print("deltaR cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["deltaR"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  

            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


            sel_dt = np.logical_and( sel_dt, np.abs(T.array('dtRechitCluster' + cluster_index + 'Size')) >= 80)
            sel_csc = np.logical_and( sel_csc, np.abs(T.array('cscRechitCluster' + cluster_index + 'Size')) >= 100)
    #         print("nrechits", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

            table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                             100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
            sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()==1, sel_dt.sum()==1))])
#             print(np.count_nonzero(np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()==1, sel_dt.sum()==1))))
#                     print(category, sel_temp)
#                     #### print start of table

    print("\\begin{table}[!h]")
    print("\\begin{center}")
    xi0 = k.split('_')[-3].replace('p','.')
    xiL = k.split('_')[-1].replace('p','.')
    p = k.split('_')[2]
    ct = k.split('_')[6][:-2]
    print("\\caption{Signal Efficiency(\\%) of each cut for "+p+" portal dark shower model, $x_{i0} = $"+xi0+", $x_{iL} = $"+xiL+", LLP mass 15 GeV and lifetime of "+ ct +" mm. \
(Cumulative Efficiency for all cuts applied after Trigger and MET cut are calculated with \
respect to events passing the MET and trigger cut and cut efficiency is calculated \
with respect to the previous cut.)}")
    print("\\label{tab:selections_sig_"+k+"}")

    print("\\begin{tabular}{c|c c | c c| cc}")
    print("\\hline")
    print(" & \\multicolumn{2}{c}{CSC-CSC} & \\multicolumn{2}{c}{DT-DT} & \\multicolumn{2}{c}{DT-CSC} \\\\")
    print("\\hline")
    print("Selection & cut eff & overall eff & cut eff & overall eff & cut eff & overall eff \\\\")
    print("\\hline")

    for k,v in table.items():
        #print table
        print(v+r" \\")
     #### print end of table
    print("\\hline")
    print("\\end{tabular}")
    print("\\end{center}")
    print("\\end{table}")
    print("\n")

\begin{table}[!h]
\begin{center}
\caption{Signal Efficiency(\%) of each cut for higgs portal dark shower model, $x_{i0} = $2.5, $x_{iL} = $1, LLP mass 15 GeV and lifetime of 5000 mm. (Cumulative Efficiency for all cuts applied after Trigger and MET cut are calculated with respect to events passing the MET and trigger cut and cut efficiency is calculated with respect to the previous cut.)}
\label{tab:selections_sig_HV_params_higgs_m_15_ctau_5000mm_xiO_2p5_xiL_1}
\begin{tabular}{c|c c | c c| cc}
\hline
 & \multicolumn{2}{c}{CSC-CSC} & \multicolumn{2}{c}{DT-DT} & \multicolumn{2}{c}{DT-CSC} \\
\hline
Selection & cut eff & overall eff & cut eff & overall eff & cut eff & overall eff \\
\hline
Trigger and MET  &  1.012 &  1.012 &  1.012 &  1.012 &  1.012 &  1.012 \\
MET filters  &  98.38 &  98.38 &  98.38 &  98.38 &  98.38 &  98.38 \\
$N_{jet} \geq 1$  &  97.44 &  95.86 &  97.44 &  95.86 &  97.44 &  95.86 \\
$N_{\text{CSC+DT rings}} \leq 10 $  & 100.00 &  95.86 & 100.00 &  95.86 & 100.00 &  9